1) Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:
InvoiceNo – номер транзакции<br>
StockCode – код товара<br>
Description – описание товара<br>
Quantity – количество единиц товара, добавленных в заказ<br>
InvoiceDate – дата транзакции <br>
UnitPrice – цена за единицу товара<br>
CustomerID – id клиента<br>
Country – страна, где проживает клиент<br>
Импортируйте данные data.csv.zip (в папке) с кодировкой ISO-8859-1. Запишите полученный датафрейм в retail, а<br> названия колонок сохраните в переменную retail_columns.

In [1]:
import pandas as pd

In [10]:
retail=pd.read_csv('data.csv.zip', compression='zip',encoding='ISO-8859-1')

In [30]:
retail.shape

(541909, 8)

In [12]:
retail_columns=retail.columns

In [13]:
retail_columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество.<br> Если они есть, то удалите их из retail.

In [25]:
retail.duplicated().value_counts()

False    536641
True       5268
dtype: int64

In [31]:
retail.drop_duplicates(inplace=True)

In [33]:
retail.shape

(536641, 8)

Данные содержат в себе записи как и об успешных транзакциях, так и об отмененных.
Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). 
Сколько всего заказов отменили пользователи?


In [37]:
retail.InvoiceNo.apply(lambda x: x.startswith('C')).value_counts()

False    527390
True       9251
Name: InvoiceNo, dtype: int64

Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0. В качестве ответа укажите число оставшихся строк.

In [40]:
retail=retail.query('Quantity>0')

In [41]:
retail.shape

(526054, 8)

Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id). Идентификатор заказа – InvoiceNo. Для каждого заказа в данных может встречаться более 1 строки.


In [42]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [78]:
ordersGermany=retail.query('Country=="Germany"')\
        .groupby('CustomerID', as_index=False)\
        .agg({'InvoiceNo':pd.Series.nunique})\
        .sort_values('InvoiceNo',ascending=False)

In [79]:
N=ordersGermany.InvoiceNo.quantile(0.8)

In [80]:
germany_top=ordersGermany.query('InvoiceNo>@N').CustomerID

In [81]:
germany_top

37    12569.0
3     12471.0
83    12720.0
52    12621.0
6     12474.0
79    12709.0
81    12712.0
64    12647.0
16    12500.0
69    12662.0
8     12476.0
12    12481.0
44    12600.0
78    12708.0
56    12626.0
77    12705.0
51    12619.0
26    12524.0
Name: CustomerID, dtype: float64

Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.


In [62]:
top_retail_germany=retail[retail.CustomerID.isin(germany_top)]

Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST? 
Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

In [72]:
top_retail_germany.query('StockCode != "POST"')\
                    .groupby(['StockCode','Description'], as_index=False)\
                    .agg({'InvoiceDate':'count'})\
                    .sort_values('InvoiceDate',ascending=False)

,StockCode,Description,InvoiceDate
469,22326,ROUND SNACK BOXES SET OF4 WOODLAND,70
471,22328,ROUND SNACK BOXES SET OF 4 FRUITS,46
519,22423,REGENCY CAKESTAND 3 TIER,42
588,22554,PLASTERS IN TIN WOODLAND ANIMALS,37
45,20719,WOODLAND CHARLOTTE BAG,37
...,...,...,...
583,22548,HEADS AND TAILS SPORTING FUN,1
585,22550,HOLIDAY FUN LUDO,1
593,22560,TRADITIONAL MODELLING CLAY,1
595,22562,MONSTERS STENCIL CRAFT,1


Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.

In [73]:
retail['Revenue']=retail['Quantity']*retail['UnitPrice']

/tmp/ipykernel_133526/1307906102.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retail['Revenue']=retail['Quantity']*retail['UnitPrice']


In [74]:
retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,16.60


Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)


In [82]:
sorted_revenuses=retail.groupby('InvoiceNo',as_index=False)\
        .agg({'Revenue':'sum'})\
        .sort_values('Revenue',ascending=False).head(5)

In [85]:
sorted_revenuses.InvoiceNo.str.cat(sep=', ')

'581483, 541431, 574941, 576365, 556444'